In [20]:
import pymongo
myclient = pymongo.MongoClient('58.16.78.136:27017')
dblist = myclient.list_database_names()
print(dblist)
mydb = myclient["test"]
collist = mydb.list_collection_names()
print(collist)

['local', 'test']
['alarm', 'item', 'system.indexes', 'test']


In [21]:
import pandas as pd
mycol = mydb["item"]
temp = []
for x in mycol.find({}, { "itemId": 1,"recordTime": 1, "value": 1 }):
    td = x
    temp.append([td.get('itemId'), td.get('recordTime'), td.get('value')])
print(len(temp))
data = pd.DataFrame(temp, columns=['itemId', 'recordTime', 'value'])

304811


In [22]:
item=list((set(list(data['itemId']))))

In [66]:
valuelist=[]
for i in item:
    valuelist.append(data[data['itemId']==i].loc[:,['value']].describe()['value'][2])

In [69]:
for i in range(len(valuelist)):
    if valuelist[i]=='null':
        valuelist[i]='0'
        

In [74]:
valuelst=list(map(float,valuelist))
valuelst

[26908774400.0,
 39752704.0,
 1015215886336.0,
 0.0,
 2408568.0,
 0.0,
 113528.0,
 0.0,
 0.0,
 0.0,
 436.0,
 32488.0,
 189081600.0,
 0.0,
 345547501568.0,
 0.0,
 39749632.0,
 28934754304.0,
 1056017055744.0,
 0.0,
 0.0,
 0.0,
 701736.0,
 0.0,
 0.0,
 2.0,
 132096.0,
 0.0,
 0.0,
 0.0,
 0.0,
 26752126976.0,
 0.0,
 822235537408.0,
 0.0,
 0.0,
 145080320.0,
 800.0,
 0.0,
 376.0,
 33734656.0,
 174824.0,
 0.0,
 0.0,
 918175744.0,
 1533677584384.0,
 0.0,
 0.0,
 197033984.0,
 0.0,
 0.0,
 0.0,
 32808.0,
 0.0,
 1007655403520.0,
 0.0,
 2.0,
 0.0,
 811.0,
 0.0,
 167.0,
 0.0,
 0.0,
 0.0,
 179696.0,
 0.0,
 152.0,
 0.0,
 626649268224.0,
 0.0,
 431.0,
 0.0,
 1.0,
 37946368.0,
 38824.0,
 0.0,
 515640.0,
 2.0,
 0.0,
 0.0,
 0.0,
 3840.0,
 0.0,
 2.0,
 0.0,
 193813504.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 514.0,
 37344.0,
 41166848.0,
 0.0,
 820785250304.0,
 0.0,
 1.0,
 0.0,
 0.0,
 194277376.0,
 828445716480.0,
 0.0,
 335.0,
 0.0,
 0.0,
 0.0,
 3018296033280.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 391624.0,
 792

In [105]:
res=[]
for i in range(len(item)):
    if valuelst[i]<=1 or valuelst[i]>10000:
        continue
    else:
        res.append(item[i])

In [119]:
len(res)

97

In [134]:
data.replace('null',100,inplace=True)

In [148]:
data421=data[data['itemId']==421]

In [152]:
data421.sort_values('recordTime')

,itemId,recordTime,value
69088,421,1558426020,99.951553
69087,421,1558426080,99.951553
69089,421,1558426140,99.951553
69090,421,1558426200,99.951553
69091,421,1558426260,99.951553
69092,421,1558426320,99.951553
69093,421,1558426380,99.951553
69094,421,1558426440,99.951553
69095,421,1558426500,99.951553
69096,421,1558426560,100


In [154]:
import pymysql
 
# 打开数据库连接
# db = pymysql.connect("58.16.78.136:30306","root",None,"predictvalue" )
d=pymysql.connect(db='aiops_monitor', user='root', passwd='1234', host='58.16.78.136', port=23306)
# 使用 cursor() 方法创建一个游标对象 cursor
cursor = d.cursor()
 
# 使用 execute()  方法执行 SQL 查询 
cursor.execute("SELECT VERSION()")
 
# 使用 fetchone() 方法获取单条数据.
da = cursor.fetchone()
 
print ("Database version : %s " % da)
 
# 关闭数据库连接
# d.close()

Database version : 8.0.13 


In [156]:
import fbprophet as fb

count=5401
for itemid in res[54:]:
    print('开始itemid为%d'% itemid)
    Fb=fb.Prophet()
    data1=data[data['itemId']==itemid].loc[:,['recordTime','value']]
    data1=data1.sort_values('recordTime')
    data1['recordTime']=pd.to_datetime(data1['recordTime'], unit='s')
    data1.columns=['ds','y']
    Fb.fit(data1)
    Pre=Fb.make_future_dataframe(freq='T',periods=100,include_history=False)
    forcasts=Fb.predict(Pre)
    forcast=forcasts.loc[:,['ds','trend','yhat_lower','yhat_upper']]
    date=list(forcast['ds'])
    trend=list(forcast['trend'])
    yhat_lower=list(forcast['yhat_lower'])
    yhat_upper=list(forcast['yhat_upper'])
    print(forcast)
    print('开始插入Itemid %d'%itemid)
    for i in range(len(date)):
        count+=1
        sql = "insert into predict(timestamp,id,value,l_value,h_value,item_id) values('%s','%d','%f','%f','%f','%d')" % (date[i],count,trend[i],yhat_lower[i],yhat_upper[i],itemid)
        try:
           # 执行SQL语句
           cursor.execute(sql)
           # 提交修改
           d.commit()
        except:
       # 发生错误时回滚
           d.rollback()

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 19.0.


开始itemid为421
                    ds      trend  yhat_lower  yhat_upper
0  2019-05-21 08:32:00  99.968314   99.948883   99.987474
1  2019-05-21 08:33:00  99.968558   99.926732  100.009904
2  2019-05-21 08:34:00  99.968801   99.888663  100.054420
3  2019-05-21 08:35:00  99.969045   99.834595  100.099796
4  2019-05-21 08:36:00  99.969289   99.773252  100.153989
5  2019-05-21 08:37:00  99.969532   99.707794  100.216004
6  2019-05-21 08:38:00  99.969776   99.637895  100.284456
7  2019-05-21 08:39:00  99.970020   99.564042  100.362166
8  2019-05-21 08:40:00  99.970263   99.481481  100.435627
9  2019-05-21 08:41:00  99.970507   99.387117  100.520741
10 2019-05-21 08:42:00  99.970750   99.300083  100.617685
11 2019-05-21 08:43:00  99.970994   99.200624  100.706740
12 2019-05-21 08:44:00  99.971238   99.106746  100.797338
13 2019-05-21 08:45:00  99.971481   99.015061  100.901378
14 2019-05-21 08:46:00  99.971725   98.881901  101.016770
15 2019-05-21 08:47:00  99.971968   98.754496  101.121781
1

InterfaceError: (0, '')

In [102]:
import time
import datetime
def timestamp_2_time(timestamp):
    temp = time.localtime(timestamp)
    temp = time.strftime("%Y-%m-%d %H:%M:%S", temp)
    time_data = pd.to_datetime(temp)
    return time_data
data['recordTime'] = data['recordTime'].apply(timestamp_2_time)
temp_date= datetime.datetime(2019, 5, 21, 11, 0, 0)
temp = data[data['recordTime'] >= temp_date]
a = temp[temp['itemId'] == 258]
print(a)

Empty DataFrame
Columns: [itemId, recordTime, value]
Index: []


In [103]:
left_date= datetime.datetime(2019, 5, 17, 0, 0, 0)
right_date = left_date + datetime.timedelta(days=1)
print(left_date, right_date)
res_data = data[(data['recordTime'] <= right_date) & (data['recordTime'] >= left_date)]
res_data

2019-05-17 00:00:00 2019-05-18 00:00:00


,itemId,recordTime,value
4,81,2019-05-17 08:55:15,38312
5,81,2019-05-17 08:56:15,38928
6,81,2019-05-17 08:57:15,38840
7,81,2019-05-17 08:58:15,38328
8,81,2019-05-17 08:59:15,38624
9,81,2019-05-17 09:00:15,39128
10,81,2019-05-17 09:01:15,38768
11,81,2019-05-17 09:02:15,38672
12,81,2019-05-17 08:55:15,38312
13,81,2019-05-17 08:56:15,38928
